In [15]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_array

from nlpbook import get_train_test_data

class BaselineClassifier(ClassifierMixin, BaseEstimator):
    def fit(self, X, y):
        y = check_array(y, ensure_2d=False)
        self.classes_, counts = np.unique(y, return_counts=True)
        self.proba_ = counts / len(y)
        return self

    def predict_proba(self, X):
        return np.repeat(self.proba_.reshape(1, self.proba_.shape[0]), len(X), axis=0)

    def predict(self, X):
        return np.full(len(X), self.classes_[np.argmax(self.proba_)])

from nlpbook import get_train_test_data

train_df, test_df = get_train_test_data()
e = BaselineClassifier().fit(train_df[['review']], train_df['label'])
e.score(test_df[['review']], test_df['label'])

0.5011190233977619